# Introduction

This notebook implements a content-based filtering recommendation system that matches a user's request to items from a product catalog using the descriptive (textual) information. 

The first step is to convert the textual item descriptions from the product catalog into numeric representations, called embeddings.  Embeddings represent the letters/numbers/phrases of the text, but also the context.  These embeddings are stored with the product catalog and referenced when a recommendation request is received.  To find items that match a request an embedding is also generated for the user's request which is compared to the embeddings from the product catalog to find similarities.  Each item in the catalog is scored based on the similarity between its embedding and the request embedding.

The next step leverages the conversational capabilties of Azure OpenAI (ChatGPT) to provide an interactive experience for user's requesting recommendations.  The prompt to ChatGPT is enriched with some of the top scoring products along with the request, then the conversation can flow to narrow down the set of recommendations to best for the specific situation for the user.



# Setup



## Prerequisites

The OpenAI python library provides access to the OpenAI API to interact with the models used for the recommender, for embeddings and conversation.  This notebook uses the Azure API type.  

**NOTE:** You will need an Azure OpenAI account endpoint and API key; to set up a new resource please see [Create an Azure OpenAI Resource](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource) or to find existing credential information, you can find this on the Azure portal on the Overview page (Endpoint and Manage Keys).

This example leverages a .env file to store the credentials used for accessing Azure OpenAI API; python-dotenv enables this.  An example file is provided with this repo called "secrets.env".  You can update this file with your information and rename it to .env to use along with this notebook.

In [1]:
#!pip install openai 
#!pip install python-dotenv

In [2]:
import os
import requests
import time

import openai
from openai.embeddings_utils import get_embedding, cosine_similarity

import pandas as pd

from dotenv import load_dotenv
load_dotenv()

True

## Azure OpenAI Models

Two types of large language models are used for the recommendation system: <br/><br/>
1) Similarity [Embeddings](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/understand-embeddings) model designed for creating the embeddings used for finding similarity between snippets of text.<br/>
2) [ChatGPT or GPT-4](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions) model for the conversational interface <br/>

This example was built using the **text-embedding-ada-002** model for similarity embeddings and **gpt-35-turbo** for the conversational functionality. 

Before continuing with this notebook, confirm that deployments exist for these types of models in your Azure OpenAI service.  

In [3]:
# set the variables to the deployment name for your models
embeddings_model_deployment = 'text-embedding-ada-002'
chat_model_deployment = 'gpt35-cm' #gpt-35-turbo

## Azure OpenAI Test

Check connectivity to Azure OpenAI Service and list all model deployments.

In [49]:
API_KEY = os.getenv("AOAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AOAI_ENDPOINT")

openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_type = "azure"
openai.api_version = '2023-03-15-preview' #"2022-12-01"

# get a list of all model deployments on the service
url = openai.api_base + "/openai/deployments?api-version=" + openai.api_version
r = requests.get(url, headers={"api-key": API_KEY})
print(r.text)

In [48]:
# Create a simple prompt
text_prompt = "Should oxford commas always be used?"

answer = openai.Completion.create(
    engine=chat_model_deployment,  
    prompt=text_prompt,
    max_tokens=60
)

print(answer['choices'][0]['text'])


 “My heroes are my parents, Nelson Mandela, and Mother Theresa” (The oxford comma is after Mandela)

Placed after Nelson Mandela, adds clarity

The oxford comma is NOT necessary in every sentence. The general rule of thumb is to determine whether it clarifies meaning and avoid ambiguity.




# Data Preperation

## Product Catalog

The product catalog consists of 298 items of retail clothing, accessories, and shoes.  The textual data available is a product title and description.

In [8]:
products_df=pd.read_csv('product_catalog.csv')
print(products_df.shape)
products_df.head()

(298, 4)


,itemId,Title,Description,Release Date
0,68719490621,Retro Horn Rimmed Keyhole Nose Bridge Round Su...,High-quality and pioneered with the perfect b...,2021-03-26T11:12:13.000Z
1,68719490622,Genuine Leather Suede Handbag Purse,This sleek Italian leather bag features an opt...,2021-06-25T11:12:13.000Z
2,68719490628,Khaki Men's Oversized Trench Coat,Lightweight design crafted with premium qualit...,2021-04-23T11:12:13.000Z
3,68719490634,Suede Leather Slip-on Loafer,Pure comfort for your feet with these handsome...,2021-02-11T11:12:13.000Z
4,68719491371,Small Calfskin Leather Satchel,"A snapping handle and secure clip, paired with...",2021-06-21T11:12:13.000Z


## Product Embeddings

The first step is to merge the fields into a single column, for convenience when generating the embeddings.  If your dataset had additional descriptive fields you could add them as well (e.g. color, sizes, categories, etc.).

In [9]:
products_df['textCombined'] = products_df.apply(lambda row: f"{row['Title']}, {row['Description']}", axis=1)
products_df.head()

,itemId,Title,Description,Release Date,textCombined
0,68719490621,Retro Horn Rimmed Keyhole Nose Bridge Round Su...,High-quality and pioneered with the perfect b...,2021-03-26T11:12:13.000Z,Retro Horn Rimmed Keyhole Nose Bridge Round Su...
1,68719490622,Genuine Leather Suede Handbag Purse,This sleek Italian leather bag features an opt...,2021-06-25T11:12:13.000Z,"Genuine Leather Suede Handbag Purse, This slee..."
2,68719490628,Khaki Men's Oversized Trench Coat,Lightweight design crafted with premium qualit...,2021-04-23T11:12:13.000Z,"Khaki Men's Oversized Trench Coat, Lightweight..."
3,68719490634,Suede Leather Slip-on Loafer,Pure comfort for your feet with these handsome...,2021-02-11T11:12:13.000Z,"Suede Leather Slip-on Loafer, Pure comfort for..."
4,68719491371,Small Calfskin Leather Satchel,"A snapping handle and secure clip, paired with...",2021-06-21T11:12:13.000Z,"Small Calfskin Leather Satchel, A snapping han..."


Next, use the embeddings model to vectorize the combined description and title text of the items into numerical representations.  For more information, see the [Embeddings REST API Reference](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#embeddings).

**NOTE:** This may take a few minutes to run.

In [10]:
#use the OpenAI method to get an embedding of the text title/description for each item in the product catalog
products_df['textEmbedding'] = products_df.textCombined.apply(lambda x: get_embedding(x, engine=embeddings_model_deployment))  

In [11]:
products_df.head(5)

,itemId,Title,Description,Release Date,textCombined,textEmbedding
0,68719490621,Retro Horn Rimmed Keyhole Nose Bridge Round Su...,High-quality and pioneered with the perfect b...,2021-03-26T11:12:13.000Z,Retro Horn Rimmed Keyhole Nose Bridge Round Su...,"[-0.016605181619524956, 0.0011062334524467587,..."
1,68719490622,Genuine Leather Suede Handbag Purse,This sleek Italian leather bag features an opt...,2021-06-25T11:12:13.000Z,"Genuine Leather Suede Handbag Purse, This slee...","[-0.0034925153013318777, 0.011994564905762672,..."
2,68719490628,Khaki Men's Oversized Trench Coat,Lightweight design crafted with premium qualit...,2021-04-23T11:12:13.000Z,"Khaki Men's Oversized Trench Coat, Lightweight...","[-0.02285175770521164, 0.003873234847560525, 0..."
3,68719490634,Suede Leather Slip-on Loafer,Pure comfort for your feet with these handsome...,2021-02-11T11:12:13.000Z,"Suede Leather Slip-on Loafer, Pure comfort for...","[0.01239150669425726, 0.012358303181827068, -0..."
4,68719491371,Small Calfskin Leather Satchel,"A snapping handle and secure clip, paired with...",2021-06-21T11:12:13.000Z,"Small Calfskin Leather Satchel, A snapping han...","[-0.008272010833024979, 0.006926483474671841, ..."


# Customer Input

The customer will enter a message, providing a information about what they're looking for, then embeddings for their search are generated.

In [12]:
customer_input = "Hi! Can you recommend some dressy shoes for me to wear with a black dress?"

In [13]:
#use the REST API to send a request to OpenAI to generate embeddings for the item(s) they are searching for.
response = openai.Embedding.create(
    input=customer_input,
    engine=embeddings_model_deployment
)
embeddings_customer_question = response['data'][0]['embedding']

# Find Similar Items

Find Similar items in the product catalog to what the user is looking for.  [Cosine Similarity](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/understand-embeddings#cosine-similarity) is used to find embeddings that are closest (more similar) to the user's search description.

In [14]:
# look for similar items in the product catalog, sort by highest searchScore
products_df['searchScore'] = products_df.textEmbedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
products_df = products_df.sort_values('searchScore', ascending=False)
products_df.head(10)

,itemId,Title,Description,Release Date,textCombined,textEmbedding,searchScore
255,68719495270,Strappy Platform Pumps,These strappy platform pumps can be paired wit...,2021-09-15T11:12:13.000Z,"Strappy Platform Pumps, These strappy platform...","[-0.013429885730147362, -0.013352850452065468,...",0.845308
249,68719495132,Pointed-toe Pumps,These red pointed-toe pumps are perfect for pa...,2021-02-16T11:12:13.000Z,"Pointed-toe Pumps, These red pointed-toe pumps...","[-0.016923705115914345, -0.029531994834542274,...",0.843851
151,68719493316,Suede Dress Shoes,Classic dress shoes paired with suede to creat...,2021-04-11T11:12:13.000Z,"Suede Dress Shoes, Classic dress shoes paired ...","[-0.004512234590947628, -0.007612389978021383,...",0.841525
243,68719494994,Platform Leather Pumps,These black platform leather pumps are perfect...,2021-01-19T11:12:13.000Z,"Platform Leather Pumps, These black platform l...","[0.007545703090727329, -0.01371828280389309, 0...",0.841292
247,68719495086,Platform Open-toe Velvet Pumps,These tan platform open-toe velvet pumps are p...,2021-04-24T11:12:13.000Z,"Platform Open-toe Velvet Pumps, These tan plat...","[-0.006942516192793846, -0.0386972613632679, 0...",0.837702
149,68719493275,Tailored Patchwork Dress Shoes,Tan dress shoes complete with tailored patchwo...,2021-01-24T11:12:13.000Z,"Tailored Patchwork Dress Shoes, Tan dress shoe...","[-0.02092052437365055, -0.017696961760520935, ...",0.836729
152,68719493343,Classic Leather Dress Shoes,Classic dress shoes made of authentic leather.,2021-08-22T11:12:13.000Z,"Classic Leather Dress Shoes, Classic dress sho...","[0.004126709885895252, -0.02312232181429863, -...",0.834452
154,68719493397,Genuine Leather Tuxedo Shoe,Black tuxedo shoe crafted with genuine leather.,2021-09-18T11:12:13.000Z,"Genuine Leather Tuxedo Shoe, Black tuxedo shoe...","[0.006228722631931305, -0.01535107847303152, -...",0.833381
198,68719494523,Strapless Black Evening Dress,This strapless black evening dress is perfect ...,2021-02-26T11:12:13.000Z,"Strapless Black Evening Dress, This strapless ...","[-0.020876901224255562, -0.007244549226015806,...",0.829093
237,68719494856,Platform Stilleto Booties,Platform stilletos are paired with black leath...,2021-05-15T11:12:13.000Z,"Platform Stilleto Booties, Platform stilletos ...","[-0.010474665090441704, -0.030687084421515465,...",0.827438


Select the top number of products to include for recommendations (e.g. top ranking based on similarity score).

In [15]:
n = 8 # number of product options to include; this will be included in the prompt - narrow down to focus, but provide enough variety to make it interesting!

topProducts= products_df.head(n)
topProducts

,itemId,Title,Description,Release Date,textCombined,textEmbedding,searchScore
255,68719495270,Strappy Platform Pumps,These strappy platform pumps can be paired wit...,2021-09-15T11:12:13.000Z,"Strappy Platform Pumps, These strappy platform...","[-0.013429885730147362, -0.013352850452065468,...",0.845308
249,68719495132,Pointed-toe Pumps,These red pointed-toe pumps are perfect for pa...,2021-02-16T11:12:13.000Z,"Pointed-toe Pumps, These red pointed-toe pumps...","[-0.016923705115914345, -0.029531994834542274,...",0.843851
151,68719493316,Suede Dress Shoes,Classic dress shoes paired with suede to creat...,2021-04-11T11:12:13.000Z,"Suede Dress Shoes, Classic dress shoes paired ...","[-0.004512234590947628, -0.007612389978021383,...",0.841525
243,68719494994,Platform Leather Pumps,These black platform leather pumps are perfect...,2021-01-19T11:12:13.000Z,"Platform Leather Pumps, These black platform l...","[0.007545703090727329, -0.01371828280389309, 0...",0.841292
247,68719495086,Platform Open-toe Velvet Pumps,These tan platform open-toe velvet pumps are p...,2021-04-24T11:12:13.000Z,"Platform Open-toe Velvet Pumps, These tan plat...","[-0.006942516192793846, -0.0386972613632679, 0...",0.837702
149,68719493275,Tailored Patchwork Dress Shoes,Tan dress shoes complete with tailored patchwo...,2021-01-24T11:12:13.000Z,"Tailored Patchwork Dress Shoes, Tan dress shoe...","[-0.02092052437365055, -0.017696961760520935, ...",0.836729
152,68719493343,Classic Leather Dress Shoes,Classic dress shoes made of authentic leather.,2021-08-22T11:12:13.000Z,"Classic Leather Dress Shoes, Classic dress sho...","[0.004126709885895252, -0.02312232181429863, -...",0.834452
154,68719493397,Genuine Leather Tuxedo Shoe,Black tuxedo shoe crafted with genuine leather.,2021-09-18T11:12:13.000Z,"Genuine Leather Tuxedo Shoe, Black tuxedo shoe...","[0.006228722631931305, -0.01535107847303152, -...",0.833381


# Setup ChatGPT Prompt

Create the message (prompt) to send to ChatGPT.  With the [Chat Completion API](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions#working-with-the-chat-completion-api) there are distinct sections of the prompt that are sent to the API in the form of an array of dictionaries with associated roles: system, user, and assistant. 

In [16]:
#start from an empty list, this will store the components of the prompts to send to the conversational model
prompt = []

Start with the system message, these are the initial instructions to the model.  This is optional, but is recommended to have at least one system instruction to get optimal results.

For our example, we will provide the set of top product recommendations to the model as context in the next steps, but we can instruct the model to use only this via the system message.

In [17]:
system_message = "You're a digital stylist helping customers by making product recommendations based on their preferences. \
You are given a question and a context. You need to answer the question using only the context.  You can provide from one to three recommendations in each answer. \
Make sure to explain your recommendations. Please be friendly and talk the user like a person don't just give a list of recommendations. "

prompt.append({"role":"system", "content": system_message})
prompt

[{'role': 'system',
  'content': "You're a digital stylist helping customers by making product recommendations based on their preferences. You are given a question and a context. You need to answer the question using only the context.  You can provide from one to three recommendations in each answer. Make sure to explain your recommendations. Please be friendly and talk the user like a person don't just give a list of recommendations. "}]

Next, we will add an example showing the format we want the results to be in.

In [18]:
# adding some special instructions and an example
prompt.append({"role":"user", "content": f"Print the itemId at the end of the recommendations enclosed in []"})
prompt.append({"role": "assistant", "content": f"I think you might like these lattice platform pumps [68719495293]."})

For a more conversational experience, try adding this to the prompt.

In [19]:
# This is a good one to add for a conversational/chat bot user interface!
#prompt.append({"role":"user", "content": f"Ask follow up questions if it would be helpful to narrow down the recommendations."})

Now we add the context, which is a number of recommended products; enough to make the conversation flow, but not so many the recommendations are low quality.

In [21]:
#add a list of top products that match the search request products_list
products_list = []

for index, row in topProducts.iterrows():
    products_list.append(f"{row['textCombined']} [{row['itemId']}]")
    
prompt.append({"role":"user", "content":f"Question: {customer_input} <context> {products_list} </context>"})
prompt

[{'role': 'system',
  'content': "You're a digital stylist helping customers by making product recommendations based on their preferences. You are given a question and a context. You need to answer the question using only the context.  You can provide from one to three recommendations in each answer. Make sure to explain your recommendations. Please be friendly and talk the user like a person don't just give a list of recommendations. "},
 {'role': 'user',
  'content': 'Print the itemId at the end of the recommendations enclosed in []'},
 {'role': 'assistant',
  'content': 'I think you might like these lattice platform pumps [68719495293].'},
 {'role': 'user',
  'content': "Question: Hi! Can you recommend some dressy shoes for me to wear with a black dress? <context> ['Strappy Platform Pumps, These strappy platform pumps can be paired with your favorite sun dress for an elegant look.  [68719495270]', 'Pointed-toe Pumps, These red pointed-toe pumps are perfect for pairing with your favo

# Get ChatGPT Recommendations

Next, the prompt is sent to the model using the [ChatCompletion REST API](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions) call which returns the recommendations.

In [22]:
completion = openai.ChatCompletion.create(
  engine=chat_model_deployment,
  messages=prompt
)

In [23]:
print(completion.choices[0].message['content'])
print(f'\nPrompt tokens: {completion["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {completion["usage"]["completion_tokens"]}')
print(f'Total tokens: {completion["usage"]["total_tokens"]}')


Of course! Based on the context you provided, there are some great options for dressy shoes to pair with a black dress! For a classic look, I would recommend either the Suede Dress Shoes [68719493316] or the Classic Leather Dress Shoes [68719493343]. Both options are timeless and versatile, which means they can be dressed up or down to fit any event.

If you're looking for something with a bit more edge, the Platform Leather Pumps [68719494994] are a great option. These black pumps have a platform sole, which provides comfort and stability while you're on your feet all night.

Another great option is the Strappy Platform Pumps [68719495270]. These strappy, black pumps also have a platform sole, and will make your legs look miles long! They pair perfectly with a black dress and add a touch of elegance to any outfit.

I hope this helps! Let me know if you have any other questions.

Prompt tokens: 371
Completion tokens: 198
Total tokens: 569


# Recommender Application

The following code puts everything together from above to facilitate testing different searches.

In [24]:
def getAOAIRecommendations(userRequest, numSearchRecs, products_df, embedding_model, chat_model):

    #embed the user's search request
    response = openai.Embedding.create(
        input=userRequest,
        engine=embedding_model
    )
    embeddings_customer_question = response['data'][0]['embedding']

    # based on the user's search, look for similar items in the product catalog, sort by highest searchScore
    products_df['searchScore'] = products_df.textEmbedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
    products_df = products_df.sort_values('searchScore', ascending=False)

    #select the top products to include for recommendations
    topProducts= products_df.head(numSearchRecs)

    #setup a list of the top products matching the user's search 
    products_list = []

    for index, row in topProducts.iterrows():
        products_list.append(f"{row['textCombined']} [{row['itemId']}]")

    prompt = []

    #setting up the personality and clear instructions for the assistant
    system_message = "You're a digital stylist helping customers by making product recommendations based on their preferences. \
    You are given a question and a context. You need to answer the question using only the context.  You can provide from one to three recommendations in each answer. \
    Make sure to explain your recommendations. Please be friendly and talk the user like a person don't just give a list of recommendations. "

    prompt.append({"role":"system", "content": system_message})

    # adding some special instructions and an example
    prompt.append({"role":"user", "content": f"Print the itemId at the end of the recommendations enclosed in []"})
    prompt.append({"role": "assistant", "content": f"I think you might like these lattice platform pumps [68719495293]."})

    # last, add the user's search question with the specific context.
    prompt.append({"role":"user", "content":f"Question: {userRequest} <context> {products_list} </context>"})

    completion = openai.ChatCompletion.create(
    engine=chat_model,
    messages=prompt,
    temperature=0.7,  #adjust as needed to make more or less random/deterministic
    )

    return completion, completion.choices[0].message['content']


In [25]:
userRequest = "Hello.  I need some shoes to wear with a black dress."
num_recs = 8 #number to include in set to select from (max 3 should be returned by prompt)

Run the following examples twice to see variations in the response and format.

In [36]:
completion, aiResponse = getAOAIRecommendations(userRequest, 10, products_df, embeddings_model_deployment, chat_model_deployment)
print(f"Customer: {userRequest}")
print(f"\nStylist: {aiResponse}")
print(f'\nPrompt tokens: {completion["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {completion["usage"]["completion_tokens"]}')
print(f'Total tokens: {completion["usage"]["total_tokens"]}')

Customer: Hello.  I need some shoes to wear with a black dress.

Stylist: Of course! For a black dress, I would suggest the following: 

1. Platform Leather Pumps [68719494994]: These black platform pumps would go perfectly with a black dress. They add height and make your legs look great, while still being comfortable enough to wear for a night out. 

2. Classic Leather Dress Shoes [68719493343]: If you're looking for something a bit more understated, these classic leather dress shoes are a perfect choice. They're made of high-quality leather and will never go out of style. 

3. Strappy Platform Pumps [68719495270]: If you're looking for a bit of fun and glam, these strappy platform pumps would be a great choice. They add height and have a fun and flirty design.

Prompt tokens: 432
Completion tokens: 160
Total tokens: 592


In [33]:
completion, aiResponse = getAOAIRecommendations(userRequest, 10, products_df, embeddings_model_deployment, chat_model_deployment)
print(f"Customer: {userRequest}")
print(f"\nStylist: {aiResponse}")
print(f'\nPrompt tokens: {completion["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {completion["usage"]["completion_tokens"]}')
print(f'Total tokens: {completion["usage"]["total_tokens"]}')

Customer: Hello.  I need some shoes to wear with a black dress.

Stylist: Great to hear that you're looking for shoes to wear with your black dress! Based on the options you've provided, I would recommend either the Platform Leather Pumps [68719494994] or the Strappy Platform Pumps [68719495270]. 

The Platform Leather Pumps are a classic choice that will pair well with any black dress. The platform design will give you some extra height and the leather material will add a touch of sophistication.

The Strappy Platform Pumps are another great option that will add some elegance to your look. The strappy design will add some visual interest and the platform heel will provide some extra height. 

Both of these options will work well with a black dress, and you can choose the one that appeals more to your personal style.

Prompt tokens: 432
Completion tokens: 156
Total tokens: 588


In [37]:
userRequest = "Hi! Can you recommend a shirt for me to wear with a suit or tuxedo?"
num_recs = 5

# The following is just to see results/options that are sent along with prompt.
#embed their request
response = openai.Embedding.create(
    input=userRequest,
    engine=embeddings_model_deployment
)
embeddings_customer_question = response['data'][0]['embedding']

# look for similar items in the product catalog, sort by highest searchScore
products_df['searchScore'] = products_df.textEmbedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
products_df = products_df.sort_values('searchScore', ascending=False)
products_df.head(num_recs)

,itemId,Title,Description,Release Date,textCombined,textEmbedding,searchScore
133,68719493034,Slim Fit Button-Down Shirt,A longsleeve slim fit black button-down shirt ...,2021-08-18T11:12:13.000Z,"Slim Fit Button-Down Shirt, A longsleeve slim ...","[-0.03441871330142021, -0.0042664045467972755,...",0.835060
135,68719493072,Casual Fit Button-Down Shirt,A longsleeve casual fit button-down white shir...,2021-01-14T11:12:13.000Z,"Casual Fit Button-Down Shirt, A longsleeve cas...","[-0.026497697457671165, -0.011109044775366783,...",0.832161
136,68719493091,Easy-Fit Button-Down Shirt,A longsleeve casual fit button-down black shir...,2021-02-23T11:12:13.000Z,"Easy-Fit Button-Down Shirt, A longsleeve casua...","[-0.03009161725640297, -0.009634483605623245, ...",0.826278
132,68719493015,Denim Button-Down Shirt,A longsleeve button-down shirt made of denim f...,2021-01-24T11:12:13.000Z,"Denim Button-Down Shirt, A longsleeve button-d...","[-0.03590767830610275, -0.010067337192595005, ...",0.819861
97,68719492334,Regular Fit Button Up Checkered Shirt,Regular fit button up checkered shirt made of ...,2021-02-11T11:12:13.000Z,"Regular Fit Button Up Checkered Shirt, Regular...","[-0.016316084191203117, -0.019890449941158295,...",0.818845


In [39]:
completion, aiResponse = getAOAIRecommendations(userRequest, 10, products_df, embeddings_model_deployment, chat_model_deployment)
print(f"Customer: {userRequest}")
print(f"\nStylist: {aiResponse}")
print(f'\nPrompt tokens: {completion["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {completion["usage"]["completion_tokens"]}')
print(f'Total tokens: {completion["usage"]["total_tokens"]}')

Customer: Hi! Can you recommend a shirt for me to wear with a suit or tuxedo?

Stylist: Certainly! Based on your request for a shirt to wear with a suit or tuxedo, I would recommend the Slim Fit Button-Down Shirt [68719493034]. This black linen shirt is slim-fitting for a modern look and will pair well with a suit or tuxedo. If you prefer a more casual look, the Plaid Long Sleeve Shirt [68719492322] could also be a great option. Its long sleeves and button-up style make it a versatile choice for dressing up or down. Finally, if you want to add a pop of color to your outfit, the Pin Striped Long Sleeve Shirt [68719492304] is a classic option that can add a touch of sophistication to your ensemble.

Prompt tokens: 454
Completion tokens: 147
Total tokens: 601
